In [20]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import datetime
import os
from time import sleep
# from tracker import *

### First Project Motion Detection 

In [2]:
cap = cv2.VideoCapture('vtest.avi') 
# frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))

# print(frame_width, frame_height)

fourcc =cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('output Motion Detection.avi',fourcc, 5.0,(1280,720))

_, frame1 = cap.read()
_, frame2 = cap.read()

while cap.isOpened():
    movement = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(movement, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray, (5,5),0)
    _, thresh = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY)
    dilation = cv2.dilate(thresh,None, iterations= 5)
    contours,_ = cv2.findContours(dilation, cv2.RETR_TREE,
                                  cv2.CHAIN_APPROX_NONE)
#     cv2.drawContours(frame1,contours,-1,(0,255,0),1)

    for contour in contours:  
        x,y,w,h = cv2.boundingRect(contour)

        if cv2.contourArea(contour) < 900:
            continue
        cv2.rectangle(frame1 , (x,y),(x+w, y+h),(0,255,0),2)
        cv2.putText(frame1, f'states: Movement',(10,30),
                    cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,(0,0,255),2)
    image = cv2.resize(frame1, (1280,720))
    output.write(image)
    cv2.imshow("gray", gray)
    cv2.imshow("dilation", dilation)
    cv2.imshow("feed", frame1)
    frame1 = frame2
    ret, frame2 = cap.read()
    if cv2.waitKey(40) == 27 or not ret:

        break  
        
cv2.destroyAllWindows()
cap.release()
output.release()

In [ ]:
# result = cv2.VideoWriter('ahmedd.mp4',
#                          cv2.VideoWriter_fourcc(*'XVID'),
#                          20, (250,250))

# tracker = EuclideanDistTracker()
# cap = cv2.VideoCapture("tracking Object videos/highway.mp4")

# object_detector = cv2.createBackgroundSubtractorMOG2(history=100,
#                                                      varThreshold=50) # if history is big number it will be hight 

# while True:

#     ret, frame = cap.read()      
#     if ret is not True:   
#         break    
#     height, width, _ = frame.shape
    
#     roi = frame[340: 720,500: 800]

#     mask = object_detector.apply(roi)
#     mask = cv2.GaussianBlur(mask, (5,5),0)
#     _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
#     mask = cv2.dilate(mask , None, iterations = 5)
    
#     contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#     detections = []
#     for cnt in contours:    
#         area = cv2.contourArea(cnt)       
#         if area > 100:       
#             x, y, w, h = cv2.boundingRect(cnt)
#             detections.append([x, y, w, h])              

#     boxes_ids = tracker.update(detections)
#     for box_id in boxes_ids:
#         x, y, w, h, id = box_id
#         cv2.putText(roi, str(id), (x, y), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
#         cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

#     cv2.imshow("roi", roi)
#     cv2.imshow("Frame", frame)
#     cv2.imshow("Mask", mask)
#     result.write(frame)
    
#     key = cv2.waitKey(30)
    
#     if key == 27:
#         break        

# result.release()
# cap.release()
# cv2.destroyAllWindows()

### Car Count

In [46]:
INTERESTED_W = 80
INTERESTED_H = 90
h_line = 550
offset = 6

cap = cv2.VideoCapture('tracking Object videos/CarStaticBackground.mp4')
fourcc= cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('countcars.mp4', fourcc, 10, (1280,720))
BGS = cv2.createBackgroundSubtractorMOG2()
count = 0
def get_center(x,y,w,h):
    w1 = int(w/2)
    h1 = int(h/2)
    center = (x+w1,y+h1)
    return center
def get_mask(img):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5),3)
    mask = BGS.apply(blur)
    dilation = cv2.dilate(mask, np.ones((3,3)))
    
    return dilation
centers = []
while True:
    ret, frame= cap.read()
    if ret:
        mask = get_mask(frame)
        contours,_ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        cv2.line(frame, (25, h_line), (1200, h_line), (176, 130, 39), 2)
        for contour in contours:
            (x,y,w,h) = cv2.boundingRect(contour)
            
#             validarea_contour = cv2.contourArea(contour) >= 1000
            validarea_contour = (w >= INTERESTED_W) and (h >= INTERESTED_H)
            if not validarea_contour:
                continue
            cv2.rectangle(frame , (x,y),(x+w,y+h), (0,255,0),2)
            center = get_center(x,y,w,h)
            centers.append(center)
            cv2.circle(frame, center,4, (0, 0, 255), -1)
            
            for x,y in centers :
                if h_line+offset > y > h_line - offset :
                    count += 1
                    cv2.line(frame, (25, h_line), (1200, h_line), (0, 127, 255), 3)
                    centers.remove((x, y)) 
                    print("No. of cars detected : " + str(count))
                
        cv2.putText(frame, "VEHICLE COUNT : "+str(count), (320, 70),cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 4)
        image = cv2.resize(frame, (1280,720))
        output.write(image)
        cv2.imshow("Video Original", frame)
        cv2.imshow(" Detectar ", mask)
        
        if cv2.waitKey(1) == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()
output.release()

No. of cars detected : 1
No. of cars detected : 2
No. of cars detected : 3
No. of cars detected : 4
No. of cars detected : 5
No. of cars detected : 6
No. of cars detected : 7
No. of cars detected : 8
No. of cars detected : 9
No. of cars detected : 10
No. of cars detected : 11
No. of cars detected : 12
